# Colorado Yeti Sim Tester
*Made by infinitempg*

For those unfamiliar with Jupyter Notebook, you need to press `Shift-Enter` in order to execute a cell. You can probably get away with just going to `Kernel -> Restart Kernel and Run All Cells`, but you should re-execute the widgets whenever you add a new folder to your directory.

In [1]:
from simFns import *
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import warnings
warnings.filterwarnings('ignore')

Make sure your entire save file folders are located in the same directory (or a sub-directory) of where this tester is located! So for example, if you copied a folder named `W9` into this directory, you would type in `./W9/`. Make sure that there is a `/` at the end of the `path`.

The `team` is the 2-3 letter acronym that can be found in the `Teams.csv` file in the `Output` folder of your save file.

In [2]:
path = './S18/W9/'
saveFile = next(os.walk(path))[1][0]
team = 'COL'
pathList = [path]

print('Loading Save File %s...'%(path+saveFile))
testrun = readSave(path,saveFile)
print('Initial Save File Loaded.')

print('\nLoading Player List for %s...'%team)
DBPlayers = testrun['playerCSV'][testrun['playerCSV']['Team'] == team]
DBPlayers['StatList'] = [statDic.get(n) for n in DBPlayers['Position']]
DBPlayers['Name'] = [n['FirstName'] + ' ' + n['LastName'] for n in DBPlayers]
DBPlayers.sort('Name')
print('Players Loaded.')

Loading Save File ./S18/W9/w8_nochanges...
Initial Save File Loaded.

Loading Player List for COL...
Players Loaded.


## Widgets

These are the widgets for testing. They are run by the dropdowns located above the graphs! The graphs are binned using the [Freedman-Diaconis Rule](https://en.wikipedia.org/wiki/Freedman%E2%80%93Diaconis_rule) for stats where the range is large, and binned by units of 1 for small ranges. If there are multiple save files, you can compare two of them together.

This first widget is for team statistics and win percentages/point differentials.

**Please note that these output Point Differentials and Win Percentages for the HOME team.** If you are the away team, make sure to adjust accordingly.

In [3]:
gameStatWidget = widgets.Dropdown(options = [n for n in testrun['gameCSV'].columns],value='PDiff')
pathWidget = widgets.Dropdown(options=pathList)
saveWidget = widgets.Dropdown(options=next(os.walk(path))[1])

def updateSaves(*args):
    saveWidget.options = next(os.walk(pathWidget.value))[1]

if len(next(os.walk(path))[1]) >= 2:
    pathWidget2 = widgets.Dropdown(options=pathList)
    saveWidget2 = widgets.Dropdown(options=next(os.walk(path))[1],value=next(os.walk(path))[1][1])
    
    def updateSaves2(*args):
        saveWidget2.options = next(os.walk(pathWidget2.value))[1]
        
    gameDisplay = interact(getGameStats,path=pathWidget,saveFile=saveWidget,plot=True,second=True,path2=pathWidget2,saveFile2=saveWidget2,stat=gameStatWidget)
    pathWidget.observe(updateSaves,'value')
    pathWidget2.observe(updateSaves2,'value')
else:
    print('No comparison file.')
    pathWidget2 = widgets.Dropdown(options=pathList)
    saveWidget2 = widgets.Dropdown(options=next(os.walk(path))[1],value=next(os.walk(path))[1][0])
    gameDisplay = interact(getGameStats,path=pathWidget,saveFile=saveWidget,plot=True,second=fixed(False),path2=fixed(pathWidget2),saveFile2=fixed(saveWidget2),stat=gameStatWidget)
    pathWidget.observe(updateSaves,'value')
    
display(gameDisplay)

interactive(children=(Dropdown(description='path', options=('./S18/W9/',), value='./S18/W9/'), Dropdown(descri…

<function simFns.getGameStats(path, saveFile, stat, plot=False, second=False, path2=None, saveFile2=None)>

### Player 

This widget is for player statistics. It may take some time to load the position-specific statistics for each player.

In [4]:
nameWidget = widgets.Dropdown(options=[n for n in DBPlayers['Name']])
playerstatWidget = widgets.Dropdown(options=[n for n in ALLstats])
playerpathWidget = widgets.Dropdown(options=pathList)
playersaveWidget = widgets.Dropdown(options=next(os.walk(path))[1])

def playerupdateSaves(*args):
    playersaveWidget.options = next(os.walk(playerpathWidget.value))[1]
    
def updateStats(*args):
    playerstatWidget.options = DBPlayers[DBPlayers['Name'] == nameWidget.value]['StatList'].data[0]

if len(next(os.walk(path))[1]) >= 2:
    playerpathWidget2 = widgets.Dropdown(options=pathList)
    playersaveWidget2 = widgets.Dropdown(options=next(os.walk(path))[1],value=next(os.walk(path))[1][1])
   
    def playerupdateSaves2(*args):
        playersaveWidget2.options = next(os.walk(playerpathWidget2.value))[1]

    playerDisplay = interact(getPlayerStats,path=playerpathWidget,saveFile=playersaveWidget,Name=nameWidget,stat=playerstatWidget,Team=fixed(team),plot=True,
                             second=True,path2=playerpathWidget2,saveFile2=playersaveWidget2)

    nameWidget.observe(updateStats,names='value')
    playerpathWidget.observe(playerupdateSaves,'value')
    playerpathWidget2.observe(updateSaves2,'value')

else:
    playerpathWidget2 = widgets.Dropdown(options=pathList)
    playersaveWidget2 = widgets.Dropdown(options=next(os.walk(path))[1],value=next(os.walk(path))[1][0])
   
    def playerupdateSaves2(*args):
        playersaveWidget2.options = next(os.walk(playerpathWidget2.value))[1]

    playerDisplay = interact(getPlayerStats,path=playerpathWidget,saveFile=playersaveWidget,Name=nameWidget,stat=playerstatWidget,Team=fixed(team),plot=True,
                             second=fixed(False),path2=fixed(playerpathWidget2),saveFile2=fixed(playersaveWidget2))

    nameWidget.observe(updateStats,names='value')
    playerpathWidget.observe(playerupdateSaves,'value')

display(playerDisplay)

interactive(children=(Dropdown(description='path', options=('./S18/W9/',), value='./S18/W9/'), Dropdown(descri…

<function simFns.getPlayerStats(path, saveFile, Name, stat, Team, plot=True, second=False, path2=None, saveFile2=None)>